In [ ]:
from pyspark.sql import SparkSession

# Create spark session on local machine
spark = SparkSession.builder \
    .master("local") \
    .config("spark.driver.bindAddress","127.0.0.1") \
    .getOrCreate()

In [ ]:
from ReviewClass import Review

spark.sparkContext.addPyFile('ReviewClass.py')

In [ ]:
import pandas as pd

# Load data from csv files into dataframe
df1 = pd.read_csv('reviews/train-1.csv')
df2 = pd.read_csv('reviews/train-2.csv')
df3 = pd.read_csv('reviews/train-3.csv')
df4 = pd.read_csv('reviews/train-4.csv')
df5 = pd.read_csv('reviews/train-5.csv')
df6 = pd.read_csv('reviews/train-6.csv')
df7 = pd.read_csv('reviews/train-7.csv')
df8 = pd.read_csv('reviews/train-8.csv')

df_train = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

# Change name of first column
df_train.rename(columns={ df_train.columns[0]: "review_id" }, inplace = True)

In [ ]:
# Add data from dataframe into spark session
reviews = [Review(**kwargs) for kwargs in df_train.to_dict(orient='records')]

review_rdd = spark.sparkContext.parallelize(reviews)

In [ ]:
# PoC of total character count
total_character_count_per_product_category_id = review_rdd\
    .flatMap(lambda review: [(review.product_category_id, review.ReviewBodyCharCount())])\
    .reduceByKey(lambda count1, count2: count1 + count2)\
    .sortByKey()\
    .collect()

print(total_character_count_per_product_category_id)